# Banco de pruebas

In [1]:
import sys
sys.path.append("../")
import re # Expresiones regulares 
import time
from utils import variables as v
import random
import requests
import pandas as pd

import matplotlib.pyplot as plt # Para visualizaciones en Jupyter no importar script python
import seaborn as sns # Para visualizaciones en Jupyter no importar script python

from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




#### Estoy intentando obtener los datos de otras stores en este caso a modo de prototipo, las de EEUU y JPN

In [2]:
# Codigo para prueba de clases que me suelte soup y pruebe todas las posibilidades
# Prueba numero de juegos
from utils import clases as c
from utils import funciones as f
from fake_useragent import UserAgent
from selenium.common.exceptions import TimeoutException
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos_es = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN","País Store"]) #,"Precio con mayor rebaja"

#Juegos store nueva, meter en codigo .py
df_juegos_us = df_juegos_es.copy()
df_juegos_jp = df_juegos_es.copy()
df_juegos = [df_juegos_es,df_juegos_us,df_juegos_jp]
lista_store = ["ESP","USA","JAP"]

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"

driver.get(link_inicial)

page = 1
cont = 19
lista_tiempo = [3,3.1,3.2]

f.carga_pagina_inicial(driver)
f.pagina_concreta_carga(page,driver)

try:
    sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont+1}]/div/a'))
    WebDriverWait(driver, timeout).until(sel_game)
except TimeoutException:
    print(f"Timed out waiting for game to appear, game number {cont}")
url = driver.current_url  
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)
soup_pagina_entera = bs(response.text,features="lxml")
url_game = soup_pagina_entera.select_one(f'[data-qa="ems-sdk-grid#productTile{cont}"] a')

# Obtenemos del href de la store española, pero también podemos obtener info de otras stores 
href_store_es = url_game.get('href')

href_store_us = href_store_es.replace('/es-es','/en-us')
href_store_jp = href_store_es.replace('/es-es','/ja-jp')

href_valor_list = [href_store_es,href_store_us,href_store_jp]

for i in range(len(href_valor_list)):

    pais_store = lista_store[i]
    
    print(href_valor_list[i])
    link_juego = link_inicial + href_valor_list[i]
    # obtenemos info del juego         
    headers = {'User-Agent': ua.random}
    response = requests.get(link_juego, headers=headers)
    soup = bs(response.text,features="lxml")

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.

    #Titulo e id del juego 
    title_info = c.info_game(soup,"h1",[{"class":"psw-m-b-5 psw-t-title-l psw-t-size-6 psw-l-line-break-word"},
                                {"class":"psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word"},
                                {"class":"psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word"}])

    id_juego = re.findall(r"\d+",href_valor_list[i])
    id_juego = int(id_juego[0])
    print(id_juego)

    titulo = title_info.caracteristica_tipo("Error al recoger el título")
    print(titulo)

    # Día y hora de webscrappeo
    fecha_webs = datetime.now()
    fecha_webs = datetime.isoformat(fecha_webs)
    print(fecha_webs)

    # Precio original sin PSN
    org_price_without_psn = c.info_game(soup,"span",[{"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},
                                                {"class":"psw-t-title-m"}])
    precio_original_sn_psn = org_price_without_psn.caracteristica_tipo("Error al recoger precio original sin psn")
    print("Precio original sin PSN" ,precio_original_sn_psn)

    # Precio original con PSN --> No aparece , solo null
    org_price_with_psn = c.info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'},
                                                {"class":"psw-t-title-s psw-c-t-2 psw-t-strike"},{"class":"psw-t-title-m"}])
    precio_original_cn_psn = org_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
    print("Precio original con PSN",precio_original_cn_psn)

    # Precio actual sin PSN
    act_price_without_psn = c.info_game(soup,"span",[{'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
    precio_actual_sn_psn = act_price_without_psn.caracteristica_tipo("Error al recoger precio actual sin psn")
    print("Precio actual sin PSN", precio_actual_sn_psn)

    # Precio actual con PSN
    act_price_with_psn = c.info_game(soup,"span",[{'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'},
                                                {'class':"psw-t-title-m psw-m-r-4"},{"class":"psw-t-title-m"}])
    precio_actual_cn_psn = act_price_with_psn.caracteristica_tipo("Error al recoger precio original con psn")
    print("Precio actual con PSN",precio_actual_cn_psn)

    #Plataforma
    pltform = c.info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#platform-value'}])

    plataforma = pltform.caracteristica_tipo("Error al recoger plataforma")
    print(plataforma)

    # Genero
    gnr = c.info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#genre-value'}])

    genero = gnr.caracteristica_tipo("Error al recoger género")
    print(genero)

    # Compañia
    company = c.info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                    'data-qa':'gameInfo#releaseInformation#publisher-value'}])

    compania = company.caracteristica_tipo("Error al recoger la compañia")
    print(compania)

    # Lanzamiento
    lanz = c.info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#releaseDate-value'}])

    lanzamiento = lanz.caracteristica_tipo("Error al recoger Lanzamiento")
    print(lanzamiento)

    # Idiomas
    lng = c.info_game(soup,"dd",[{'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max',
                                'data-qa':'gameInfo#releaseInformation#subtitles-value'}])

    idiomas = lng.caracteristica_tipo("Error al recoger Idiomas")
    print(idiomas)

    # Nº de calificaciones

    num_cal = c.info_game(soup,"span",[{'class':'psw-c-t-2 psw-t-secondary',
                                    'data-qa':'mfe-star-rating#overall-rating#total-ratings'}])

    num_calificaciones = num_cal.caracteristica_tipo("Error al recoger num calificaciones")
    print(num_calificaciones)

    # Calificación PSN
    cal_psn = c.info_game(soup,"div",[{'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}])

    calificacion = cal_psn.caracteristica_tipo("Error al recoger Calificación PSN")
    print(calificacion)


    # Calificaciones por estrellas

    cal_stars = c.info_game(soup,"span",[{'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'},
                                {'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}])

    list_stars = cal_stars.caracteristica_tipo("Error al obtener calificaciones por estrella",cal=True)
    calificacion_1 = list_stars[0]
    calificacion_2 = list_stars[1]
    calificacion_3 = list_stars[2]
    calificacion_4 = list_stars[3]
    calificacion_5 = list_stars[4]
    
    print(calificacion_1,calificacion_2,calificacion_3,calificacion_4,calificacion_5)
    
    df_juegos[i].loc[len(df_juegos[i])] = {"id_juego":id_juego,"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                        "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                        "Calificación 5 estrellas":calificacion_5,
                                        "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                        "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                        "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn, "País Store":pais_store}
  
df_juegos[0]
df_juegos[1]
df_juegos[2]

/es-es/concept/10002313
10002313
The Crew™ Motorfest Standard Edition - Cross-Gen Bundle
2024-04-02T19:01:20.979725
Precio original sin PSN 89,99 €
Precio original con PSN 89,99 €
Precio actual sin PSN 44,99 €
Precio actual con PSN 44,99 €
PS4, PS5
Conducción/Carreras
UBISOFT ENTERTAINMENT SA
14/9/2023
Alemán, Español, Francés (Francia), Inglés, Italiano, Polaco, Portugués (Brasil)
18396 calificaciones
4.28
10 % 2 % 6 % 13 % 69 %
/en-us/concept/10002313
10002313
The Crew™ Motorfest Standard Edition - Cross-Gen Bundle
2024-04-02T19:01:24.739287
Precio original sin PSN $79.99
Precio original con PSN $79.99
Precio actual sin PSN $39.99
Precio actual con PSN $39.99
PS4, PS5
Driving/Racing
Ubisoft Entertainment
9/14/2023
English, French (France), Portuguese (Brazil), Spanish
18396 ratings
4.28
10% 2% 6% 13% 69%
/ja-jp/concept/10002313
10002313
ザ・クルー：モーターフェス スタンダードエディション - Cross-Genバンドル
2024-04-02T19:01:28.806117
Precio original sin PSN ¥11,220
Precio original con PSN ¥11,220
Precio actual s

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,ザ・クルー：モーターフェス スタンダードエディション - Cross-Genバンドル,2024-04-02T19:01:28.806117,"PS4, PS5",ドライビング／レース,ユービーアイソフト(株),2023/9/11,"中国語 (簡体字), 中国語 (繁体字), 日本語, 英語, 韓国語",4.28,18396件の評価,69％,13％,6％,2％,10％,"¥11,220","¥5,610","¥11,220","¥5,610",JAP


In [ ]:
df_juegos[0]

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,Marvel’s Spider-Man 2,2024-03-10T23:11:46.831501,PS5,Acción,Sony Interactive Entertainment Europe,19/10/2023,"Alemán, Checo, Danés, Español, Finlandés, Fran...",4.88,109.089 calificaciones,93 %,4 %,1 %,1 %,1 %,"79,99 €","79,99 €","79,99 €","79,99 €",ESP


In [ ]:
df_juegos[1]

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN,País Store
0,Marvel’s Spider-Man 2,2024-03-10T23:11:50.772913,PS5,Action,Sony Interactive Entertainment,10/20/2023,"English, French (France), Portuguese (Brazil),...",4.88,"109,089 ratings",93%,4%,1%,1%,1%,$69.99,$69.99,$69.99,$69.99,USA


In [ ]:
href_valor_list

['/es-es/concept/10002456',
 '/en-us/concept/10002456',
 '/ja-jp/concept/10002456']

Cosas a meter nuevas en variables

In [ ]:
# Lista de recheck
lista_recheck = []


df_juegos_us = df_juegos_es.copy()
df_juegos_jp = df_juegos_es.copy()
df_juegos = [df_juegos_es,df_juegos_us,df_juegos_jp]
lista_store = ["ESP","USA","JAP"]

Lo siguiente en la obtención del href de cada store para después meter un for pasando por cada uno, haciendo una sopa y recopilado la información concreta

In [ ]:
href_store_es = url_game.get('href')

href_store_us = href_store_es.replace('/es-es','/en-us')
href_store_jp = href_store_es.replace('/es-es','/ja-jp')

href_valor_list = [href_store_es,href_store_us,href_store_jp]

In [3]:
soup

<!DOCTYPE html>
<html class="psw-root psw-rem-base" dir="ltr" lang="ja-jp"><head><meta content="3.1.0" name="wca-ssr-version"/><meta content="ja-JP" name="wca-locale"/><meta content="dark" name="wca-theme"/><script id="wca-config-overrides" type="application/json">
    {"client":{"webCheckout":{"clientId":"2eb25762-877f-4140-b341-7c7e14c19f98"}}}</script>
<link crossorigin="" href="https://static.playstation.com" rel="preconnect"/>
<link href="https://static.playstation.com" rel="dns-prefetch"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="preconnect"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="dns-prefetch"/>
<link crossorigin="" href="https://image.api.playstation.com" rel="preconnect"/>
<link href="https://image.api.playstation.com" rel="dns-prefetch"/>
<link href="https://id.sonyentertainmentnetwork.com" rel="dns-prefetch"/>
<link href="https://checkout.playstation.com" rel="dns-prefetch"/>
<link href="https://static.playstation.com/font-sta

buscando mejores formas de extraer datos con diccionarios desde un solo boton

In [6]:
dict_completo = soup.find('button',attrs={'data-qa':'mfeCtaMain#cta#action'}).get_attribute_list('data-telemetry-meta')[0]

In [7]:
import json
conv_json = json.loads(dict_completo)

Titulo  
Día y hora  
Plataforma  
Genero  
Compañia  
Lanzamiento  
Idiomas  
Calificación PSN  
Número de calificaciones  
Calificación 5 estrellas  
Calificación 4 estrellas  
Calificación 3 estrellas  
Calificación 2 estrellas  
Calificación 1 estrella  
Precio original sin PSN	  
Precio actual sin PSN  
Precio original con PSN  
Precio actual con PSN  
País Store

In [8]:
conv_json

{'conceptId': '10002313',
 'ctaIndex': 0,
 'ctaSubType': 'add_to_library',
 'ctaType': 'ADD_TO_LIBRARY',
 'interactAction': 'click add to library',
 'interactCta': 'add to library',
 'productId': 'JP0001-PPSA02840_00-THECREWORL000000',
 'productDetail': [{'productId': 'JP0001-PPSA02840_00-THECREWORL000000',
   'productName': 'ザ・クルー：モーターフェス スタンダードエディション - Cross-Genバンドル',
   'productToSkuIds': ['JP0001-PPSA02840_00-THECREWORL000000-J002',
    'JP0001-PPSA02840_00-THECREWORL000000-J007'],
   'productPriceDetail': [{'rewardId': '',
     'offerCampaignId': None,
     'originalPriceFormatted': '無料',
     'originalPriceValue': 0,
     'discountPriceFormatted': '無料',
     'discountPriceValue': 0,
     'priceCurrencyCode': 'JPY',
     'offerBranding': 'NONE',
     'offerQualification': [],
     'offerExclusive': False,
     'offerApplied': False,
     'offerApplicability': 'APPLICABLE',
     'offerIsTiedToSubscription': False},
    {'rewardId': '',
     'offerCampaignId': None,
     'originalPr

Poner el tipo de descuento y porque, no solo plataforma PSN

In [ ]:
conv_json['conceptId'] # ID

'205354'

In [ ]:
conv_json['productId']

'JP9000-PPSA05685_00-UNCHARTED4R00000'

In [ ]:
conv_json['productDetail'][0]

{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
 'productName': 'アンチャーテッド トレジャーハンターコレクション',
 'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
  'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
 'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
   'offerCampaignId': 'PROMO-PROD-00007636',
   'originalPriceFormatted': '¥5,390',
   'originalPriceValue': 5390,
   'discountPriceFormatted': '¥2,156',
   'discountPriceValue': 2156,
   'priceCurrencyCode': 'JPY',
   'offerBranding': 'NONE',
   'offerQualification': [],
   'offerExclusive': False,
   'offerApplied': True,
   'offerApplicability': 'APPLICABLE',
   'offerIsTiedToSubscription': False},
  {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
   'offerCampaignId': None,
   'originalPriceFormatted': '¥5,390',
   'originalPriceValue': 5390,
   'discountPriceFormatted': '含まれます',
   'discountPriceValue': 0,
   'priceCurrencyCode': 'JPY',
   'offerBranding': 'PS_PLUS',
   'offerQualification': [{'__typename': 'Q

In [ ]:
conv_json['productDetail'][0]['productPriceDetail'][1]['originalPriceFormatted'] # Precio original

'¥5,390'

In [ ]:
conv_json['productDetail'][0]['productPriceDetail'][1]['discountPriceValue'] # Precio actual con PSN u otro servicio

0

In [ ]:
conv_json['productDetail'][0]['productPriceDetail'][1]['offerBranding'] # Motivo de oferta, campaña

'PS_PLUS'

In [ ]:
conv_json['productDetail'][0]['productPriceDetail'][0]['originalPriceFormatted'] # Precio original sin PSN

'¥5,390'

In [ ]:
conv_json['productDetail'][0]['productPriceDetail'][0]['discountPriceFormatted'] # Precio actual sin PSN

'¥2,156'

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

In [ ]:
conv_json['productDetail']

[{'productId': 'JP9000-PPSA05685_00-UNCHARTED4R00000',
  'productName': 'アンチャーテッド トレジャーハンターコレクション',
  'productToSkuIds': ['JP9000-PPSA05685_00-UNCHARTED4R00000-J002',
   'JP9000-PPSA05685_00-UNCHARTED4R00000-J003'],
  'productPriceDetail': [{'rewardId': 'OFFER-PROD-7636_60',
    'offerCampaignId': 'PROMO-PROD-00007636',
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '¥2,156',
    'discountPriceValue': 2156,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'NONE',
    'offerQualification': [],
    'offerExclusive': False,
    'offerApplied': True,
    'offerApplicability': 'APPLICABLE',
    'offerIsTiedToSubscription': False},
   {'rewardId': 'SUB001-SUBC00002_00-HM93SKR5MGPG6TD8',
    'offerCampaignId': None,
    'originalPriceFormatted': '¥5,390',
    'originalPriceValue': 5390,
    'discountPriceFormatted': '含まれます',
    'discountPriceValue': 0,
    'priceCurrencyCode': 'JPY',
    'offerBranding': 'PS_PLUS',
    'offerQualific

codigo EA offer

<button data-qa="mfeCtaMain#cta#action" data-track="{&quot;eventInfo&quot;:{&quot;eventAction&quot;:&quot;click&quot;,&quot;eventName&quot;:&quot;add to cart&quot;},&quot;attributes&quot;:{&quot;moduleName&quot;:&quot;pricing cta&quot;,&quot;assetName&quot;:&quot;&quot;,&quot;label&quot;:&quot;add to cart&quot;,&quot;position&quot;:&quot;1&quot;,&quot;linkUrl&quot;:&quot;&quot;,&quot;elementType&quot;:&quot;button&quot;,&quot;userActionFlag&quot;:false,&quot;sku&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;}}" data-track-click="ctaWithPrice:addToCart" data-telemetry-meta="{&quot;conceptId&quot;:&quot;10002958&quot;,&quot;ctaIndex&quot;:0,&quot;ctaSubType&quot;:&quot;add_to_cart&quot;,&quot;ctaType&quot;:&quot;ADD_TO_CART&quot;,&quot;interactAction&quot;:&quot;click add to cart&quot;,&quot;interactCta&quot;:&quot;add to cart&quot;,&quot;productId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000&quot;,&quot;productDetail&quot;:[{&quot;productId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000&quot;,&quot;productName&quot;:&quot;Dead Space&quot;,&quot;productToSkuIds&quot;:[&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;],&quot;productPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:7999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false},{&quot;rewardId&quot;:&quot;EP5679-CUSA15082_00-RWD-8571459&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;Incluido&quot;,&quot;discountPriceValue&quot;:0,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;EA_ACCESS&quot;,&quot;offerQualification&quot;:[{&quot;__typename&quot;:&quot;Qualification&quot;,&quot;type&quot;:&quot;ENTITLEMENT_IN_CART&quot;,&quot;value&quot;:&quot;EP5679-CUSA15082_00-RWD-8571459&quot;}],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;UPSELL&quot;,&quot;offerIsTiedToSubscription&quot;:true}]}],&quot;skuDetail&quot;:[{&quot;skuId&quot;:&quot;EP0006-PPSA03846_00-BEYONDPS5GAME000-E003&quot;,&quot;skuPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:7999,&quot;discountPriceFormatted&quot;:&quot;79,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:7999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false}],&quot;skuName&quot;:&quot;Juego&quot;}],&quot;titleId&quot;:&quot;PPSA03846_00&quot;}" id="" class="psw-fill-x dtm-track psw-button psw-b-0 psw-t-button psw-l-line-center psw-button-sizing psw-button-sizing--medium psw-purchase-button psw-solid-button" tabindex="0" role="" type="button" name=""><span class="psw-fill-x psw-t-truncate-1 psw-l-space-x-2 ">Añadir al carrito</span></button>

codigo psn offer

<button data-qa="mfeCtaMain#cta#action" data-track="{&quot;eventInfo&quot;:{&quot;eventAction&quot;:&quot;click&quot;,&quot;eventName&quot;:&quot;add to cart&quot;},&quot;attributes&quot;:{&quot;moduleName&quot;:&quot;pricing cta&quot;,&quot;assetName&quot;:&quot;&quot;,&quot;label&quot;:&quot;add to cart&quot;,&quot;position&quot;:&quot;1&quot;,&quot;linkUrl&quot;:&quot;&quot;,&quot;elementType&quot;:&quot;button&quot;,&quot;userActionFlag&quot;:false,&quot;sku&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;}}" data-track-click="ctaWithPrice:addToCart" data-telemetry-meta="{&quot;conceptId&quot;:&quot;205354&quot;,&quot;ctaIndex&quot;:0,&quot;ctaSubType&quot;:&quot;add_to_cart&quot;,&quot;ctaType&quot;:&quot;ADD_TO_CART&quot;,&quot;interactAction&quot;:&quot;click add to cart&quot;,&quot;interactCta&quot;:&quot;add to cart&quot;,&quot;productId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000&quot;,&quot;productDetail&quot;:[{&quot;productId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000&quot;,&quot;productName&quot;:&quot;UNCHARTED: Colección Legado de los Ladrones&quot;,&quot;productToSkuIds&quot;:[&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;],&quot;productPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;OFFER-PROD-7634_60&quot;,&quot;offerCampaignId&quot;:&quot;PROMO-PROD-00007634&quot;,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;19,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:1999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:true,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false},{&quot;rewardId&quot;:&quot;SUB001-SUBC00002_00-TJ4KZ94GWFC3NYB0&quot;,&quot;offerCampaignId&quot;:null,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;Incluido&quot;,&quot;discountPriceValue&quot;:0,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;PS_PLUS&quot;,&quot;offerQualification&quot;:[{&quot;__typename&quot;:&quot;Qualification&quot;,&quot;type&quot;:&quot;ENTITLEMENT_IN_CART&quot;,&quot;value&quot;:&quot;SUB001-SUBC00002_00-TJ4KZ94GWFC3NYB0&quot;}],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:false,&quot;offerApplicability&quot;:&quot;UPSELL&quot;,&quot;offerIsTiedToSubscription&quot;:true}]}],&quot;skuDetail&quot;:[{&quot;skuId&quot;:&quot;EP9000-PPSA05684_00-UNCHARTED4R00000-E003&quot;,&quot;skuPriceDetail&quot;:[{&quot;rewardId&quot;:&quot;OFFER-PROD-7634_60&quot;,&quot;offerCampaignId&quot;:&quot;PROMO-PROD-00007634&quot;,&quot;originalPriceFormatted&quot;:&quot;49,99&nbsp;€&quot;,&quot;originalPriceValue&quot;:4999,&quot;discountPriceFormatted&quot;:&quot;19,99&nbsp;€&quot;,&quot;discountPriceValue&quot;:1999,&quot;priceCurrencyCode&quot;:&quot;EUR&quot;,&quot;offerBranding&quot;:&quot;NONE&quot;,&quot;offerQualification&quot;:[],&quot;offerExclusive&quot;:false,&quot;offerApplied&quot;:true,&quot;offerApplicability&quot;:&quot;APPLICABLE&quot;,&quot;offerIsTiedToSubscription&quot;:false}],&quot;skuName&quot;:&quot;Juego&quot;}],&quot;titleId&quot;:&quot;PPSA05684_00&quot;}" id="" class="psw-fill-x dtm-track psw-button psw-b-0 psw-t-button psw-l-line-center psw-button-sizing psw-button-sizing--medium psw-purchase-button psw-solid-button" tabindex="0" role="" type="button" name=""><span class="psw-fill-x psw-t-truncate-1 psw-l-space-x-2 ">Añadir al carrito</span></button>

[221941, 10001023, 205132, 235080, 202007, 202523, 224832, 229776, 227255, 203706, 202665, 204923, 202006, 208240, 234597, 10010335, 10007334, 232640, 10007962, 10001257, 10001949, 10008926, 10009430, 10008712, 201386, 10003393, 10007921, 10008377, 10010438, 232136, 201687, 232798, 10007580, 10007315, 222257, 10008940, 10008416, 10009665, 232230, 10009252, 10008403, 10005702, 10000500, 10003237, 202939, 10008737, 10008487, 10009778, 10009509, 229436, 10007261, 10009541, 10010442, 10005760, 216508, 10009421, 10009831, 216554, 10008557, 10008524, 10009752, 10000976, 10009242, 10004289, 10009679, 10002260, 10002211, 205520]


Pruebas de carga en FastApi

In [ ]:
import sys
sys.path.append("../")
# Prueba de codigo dentro de la API
from utils import funciones as f
from utils import clases as c
from utils import variables as v
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json


# Cargamos página inicial
driver,service,options = f.carga_driver()
driver.get(v.link_inicial)
f.carga_pagina_inicial(driver)

# Cargamos cuadro de busqueda
search_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/header/div/section/span/span/button'))
WebDriverWait(driver, v.timeout).until(search_)
search = driver.find_element(By.XPATH,'/html/body/div[2]/header/div/section/span/span/button')
search.click()

barra_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div/div/div/div[2]/input'))
WebDriverWait(driver, v.timeout).until(barra_)
barra_busqueda = driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/div[2]/input')
barra_busqueda.send_keys("resident evil")
click_barra = driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div/button[2]')
click_barra.click()

try:
    time.sleep(1.5)
    sel_game = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/section/div/ul/li[1]/div/a/div/div/div[1]/span[2]/img[2]'))
    WebDriverWait(driver, v.timeout).until(sel_game)
    enl_game = driver.find_element(By.XPATH, '/html/body/div[3]/main/section/div/ul/li[1]/div/a/div/div/div[1]/span[2]/img[2]')
    enl_game.click()
except TimeoutException:
    print(f"Timed out waiting for game to appear, game number {v.game}, número de intentos {v.intentos}")

dict_completo = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@data-qa='mfeCtaMain#cta#action']")))
data_telemetry_meta = dict_completo.get_attribute('data-telemetry-meta')
id_game = json.loads(data_telemetry_meta)['conceptId']

driver.quit() 

232112


Pruebas Cambio de moneda

In [14]:
import sys
sys.path.append("../")
# Prueba de codigo dentro de la API
from utils import funciones as f
from utils import clases as c
from utils import variables as v
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json

driver,service,options = f.carga_driver()
driver.get(v.link_google)
barra_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea'))
WebDriverWait(driver, v.timeout).until(barra_)
barra_busqueda = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea')
barra_busqueda.send_keys('yen')
next_page_number = EC.presence_of_all_elements_located((By.XPATH,'/html/body/div[6]/div/div[12]/div[1]/div[2]/div[2]/div/div/div[1]/div/block-component/div/div[1]/div/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]'))
dato = next_page_number.get_attribute()
print({"Cambio_moneda_'yen'}": dato})

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF610A1AD22+56930]
	(No symbol) [0x00007FF61098F622]
	(No symbol) [0x00007FF61084419D]
	(No symbol) [0x00007FF61083239D]
	(No symbol) [0x00007FF610830676]
	(No symbol) [0x00007FF610830EDC]
	(No symbol) [0x00007FF6108469FD]
	(No symbol) [0x00007FF6108C8B47]
	(No symbol) [0x00007FF6108ABC9A]
	(No symbol) [0x00007FF6108C81E2]
	(No symbol) [0x00007FF6108ABA43]
	(No symbol) [0x00007FF61087D438]
	(No symbol) [0x00007FF61087E4D1]
	GetHandleVerifier [0x00007FF610D96AAD+3709933]
	GetHandleVerifier [0x00007FF610DEFFED+4075821]
	GetHandleVerifier [0x00007FF610DE817F+4043455]
	GetHandleVerifier [0x00007FF610AB9756+706710]
	(No symbol) [0x00007FF61099B8FF]
	(No symbol) [0x00007FF610996AE4]
	(No symbol) [0x00007FF610996C3C]
	(No symbol) [0x00007FF6109868F4]
	BaseThreadInitThunk [0x00007FFB0B0D7344+20]
	RtlUserThreadStart [0x00007FFB0BAE26B1+33]
